In [1]:
#IMPORTS
import numpy as np
import pydicom as dicom
from pydicom.fileset import FileSet
import pathlib
import os
import glob
import shutil
#https://medium.com/p/f1623a7f40b8
#https://gist.githubusercontent.com/alex-weston-13/4dae048b423f1b4cb9828734a4ec8b83/raw/59e49d168421a667645c9ab231d3993fe0752724/sort_dicoms.py

In [2]:
dicomfilepath = pathlib.PureWindowsPath('C:\\Users\\shoun\\OneDrive - TUM\\Downloads\\patientdata_yr_19\\19\\0001074223\\DICOMDIR')
# dicomfilepath_iflpc = pathlib.PurePosixPath('/mnt/projects/DeepProstateDB/Data/14/0001484378/DICOMDIR')
ds = dicom.dcmread(dicomfilepath)
# ds = dicom.dcmread(dicomfilepath_iflpc)  #ds stores the path to dicom directory
print(ds)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [4]:
file = FileSet(ds)
print(file)
#shows the dicom file as a file set

DICOM File-set
  Root directory: C:\Users\shoun\OneDrive - TUM\Downloads\patientdata_yr_19\19\0001074223
  File-set ID: SECTRA FILE SET
  File-set UID: 1.2.840.10008.123456.730961.2016008521.485766558.1311904132
  Descriptor file ID: (no value available)
  Descriptor file character set: ISO_IR 100
  Changes staged for write(): DICOMDIR update, directory structure update

  Managed instances:
    PATIENT: PatientID='ANON8JMHEF112', PatientName='Anonymous'
      STUDY: StudyDate=20190130, StudyTime=102013.000000, StudyDescription='MRT Becken nativ und mit KM'
        SERIES: Modality=MR, SeriesNumber=1
          IMAGE: 7 SOP Instances
        SERIES: Modality=MR, SeriesNumber=2
          IMAGE: 1 SOP Instance
        SERIES: Modality=MR, SeriesNumber=3
          IMAGE: 20 SOP Instances
        SERIES: Modality=MR, SeriesNumber=4
          IMAGE: 20 SOP Instances
        SERIES: Modality=MR, SeriesNumber=5
          IMAGE: 20 SOP Instances
        SERIES: Modality=MR, SeriesNumber=6
     

In [5]:
#This cell reads the entire directory, and prints the files also returning tne file count
#for windows
src = "C:\\Users\\shoun\\OneDrive - TUM\\Downloads\\patientdata_yr_19\\19\\0001074223\\DICOM\\"
dst = "C:\\Users\\shoun\\OneDrive - TUM\\Projects\\outputs\\digibiop\\"

#for linux iflpc
# src = "/mnt/projects/DeepProstateDB/Data/19/0002208807/DICOM/0000A326/AA1740DD/AA0ABBD7/0000A9DF"
# dst = "/mnt/HDD1/shounak/test"
print('reading file list...')
unsortedList = []
for root, dirs, files in os.walk(src):
    print("root",root)
    print("dirs",dirs)
    print("files",files)
    for file in files: 
#         if ".file" in file:# exclude non-dicoms, good for messy folders
            unsortedList.append(os.path.join(root, file))

print('%s files found.' % len(unsortedList))

reading file list...
root C:\Users\shoun\OneDrive - TUM\Downloads\patientdata_yr_19\19\0001074223\DICOM\
dirs ['00008EE4']
files []
root C:\Users\shoun\OneDrive - TUM\Downloads\patientdata_yr_19\19\0001074223\DICOM\00008EE4
dirs ['AAF2E2CF']
files []
root C:\Users\shoun\OneDrive - TUM\Downloads\patientdata_yr_19\19\0001074223\DICOM\00008EE4\AAF2E2CF
dirs ['AA12F9DA']
files []
root C:\Users\shoun\OneDrive - TUM\Downloads\patientdata_yr_19\19\0001074223\DICOM\00008EE4\AAF2E2CF\AA12F9DA
dirs ['00000505', '0000178C', '00001A27', '00001E4E', '00001EBB', '0000252B', '0000359C', '000035FB', '00003A84', '0000525B', '000053E2', '000065A9', '00006AC3', '00007383', '00008832', '0000931D', '00009C12', '0000C126', '0000C1A0', '0000E633', '0000EC4C', '0000EEB7', '0000F143', '0000FAA4']
files []
root C:\Users\shoun\OneDrive - TUM\Downloads\patientdata_yr_19\19\0001074223\DICOM\00008EE4\AAF2E2CF\AA12F9DA\00000505
dirs []
files ['EE03BF8F', 'EE1E193D', 'EE1E899C', 'EE2B5057', 'EE2F979C', 'EE3834FE', 'E

In [6]:
testfile = dicom.dcmread('C:\\Users\\shoun\\OneDrive - TUM\\Downloads\\patientdata_yr_19\\19\\0001074223\\DICOM\\00008EE4\\AAF2E2CF\\AA12F9DA\\0000C1A0\\EE0C0607')
print(testfile[0x0008, 0x103e]) #testfile for testing

(0008, 103e) Series Description                  LO: 'Prostate_t1_twist_tra_dyn_fs_SUB_MIP_TRA'


In [7]:
#EXCLUDE PUNCTUATIONS

def clean_text(string):
    # clean and standardize text descriptions, which makes searching files easier
    forbidden_symbols = ["*", ".", ",", "\"", "\\", "/", "|", "[", "]", ":", ";", " "]
    for symbol in forbidden_symbols:
        string = string.replace(symbol, "_") # replace everything with an underscore
    return string.lower()

In [11]:
for dicom_loc in unsortedList:
    # read the file
    ds = dicom.read_file(dicom_loc, force=True)
    
    # get patient, study, and series information
    patientID = clean_text(ds.get("PatientID", "NA"))
    studyDate = clean_text(ds.get("StudyDate", "NA"))
    studyDescription = clean_text(ds.get("StudyDescription", "NA"))
    seriesDescription = clean_text(ds.get("SeriesDescription", "NA"))
#     print(type(ds[0x0008, 0x103e])) #<class 'pydicom.dataelem.DataElement'>
    # generate new, standardized file name
    modality = ds.get("Modality","NA")
    studyInstanceUID = ds.get("StudyInstanceUID","NA")
#     seriesInstanceUID = ds.get("SeriesInstanceUID","NA")
    instanceNumber = str(ds.get("InstanceNumber","0"))
    fileName = modality + "." + seriesDescription + "." + instanceNumber + '.dcm'
#     fileName = modality + "." + seriesDescription + "." + instanceNumber + .dcm
    print(fileName)
    # save files to a 4-tier nested folder structure
    if not os.path.exists(os.path.join(dst, patientID)):
            os.makedirs(os.path.join(dst, patientID))
    if not os.path.exists(os.path.join(dst, patientID, studyDate)):
            os.makedirs(os.path.join(dst, patientID, studyDate))
    if not os.path.exists(os.path.join(dst, patientID, studyDate, studyDescription)):
            os.makedirs(os.path.join(dst, patientID, studyDate, studyDescription))
    if not os.path.exists(os.path.join(dst, patientID, studyDate, studyDescription, seriesDescription)):
            os.makedirs(os.path.join(dst, patientID, studyDate, studyDescription, seriesDescription))
            print('Saving out file: %s - %s - %s - %s.' % (patientID, studyDate, studyDescription, seriesDescription ))
    ds.save_as(os.path.join(dst, patientID, studyDate, studyDescription, seriesDescription, fileName))

print('done.')
    

MR.prostate_diff_b800.19.dcm
Saving out file: anon8jmhef112 - 20190130 - mrt_becken_nativ_und_mit_km - prostate_diff_b800.
MR.prostate_diff_b800.12.dcm
MR.prostate_diff_b800.6.dcm
MR.prostate_diff_b800.20.dcm
MR.prostate_diff_b800.16.dcm
MR.prostate_diff_b800.11.dcm
MR.prostate_diff_b800.10.dcm
MR.prostate_diff_b800.13.dcm
MR.prostate_diff_b800.7.dcm
MR.prostate_diff_b800.17.dcm
MR.prostate_diff_b800.2.dcm
MR.prostate_diff_b800.4.dcm
MR.prostate_diff_b800.18.dcm
MR.prostate_diff_b800.9.dcm
MR.prostate_diff_b800.5.dcm
MR.prostate_diff_b800.15.dcm
MR.prostate_diff_b800.1.dcm
MR.prostate_diff_b800.8.dcm
MR.prostate_diff_b800.14.dcm
MR.prostate_diff_b800.3.dcm
PT.prostate_mip.20.dcm
Saving out file: anon8jmhef112 - 20190130 - mrt_becken_nativ_und_mit_km - prostate_mip.
PT.prostate_mip.27.dcm
PT.prostate_mip.1.dcm
PT.prostate_mip.16.dcm
PT.prostate_mip.14.dcm
PT.prostate_mip.17.dcm
PT.prostate_mip.24.dcm
PT.prostate_mip.7.dcm
PT.prostate_mip.29.dcm
PT.prostate_mip.12.dcm
PT.prostate_mip.15.

In [47]:
# def search_file(directory = None, file = None):
#     assert os.path.isdir(directory)
#     for cur_path, directories, files in os.walk(directory):
#         if file in files:
#             return os.path.join(directory, cur_path, file)
#     return None
    

In [80]:
# folder = "/mnt/HDD1/shounak/test"
folder ="/mnt/HDD1/shounak/Biopsy_output_year_sorted/17"

# sub_folders = [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]

# print(sub_folders)

# for root, dirs, files in os.walk(folder, topdown=False):
#     for name in dirs:
#         print (os.path.join(root, name))

# listdir = os.listdir(folder)
# print(listdir)

# for item in folder:

#     if os.path.isdir(item):

#         print(item)

# print (glob.glob("/mnt/HDD1/shounak/Biopsy_output_year_sorted/17/*"))
# print(search_file("/mnt/HDD1/shounak/test", None))
dest = "/mnt/HDD1/shounak/test/testforcopyingfiles/"

for name in glob.glob('/mnt/HDD1/shounak/Biopsy_output_year_sorted/17/*/*/*/prostate_diff_b800/'):
#     print ('\t', type(name)) #<class 'str'>
#     print ('\t', (name))
    path_src = pathlib.Path(name)
#     print(type(path_src))
    full_file_name = os.path.join(path_src, file_name)
    print(full_file_name)
    if os.path.isfile(full_file_name):
        shutil.copy(full_file_name, dest)
#         src_files = os.listdir(path_src)
#     print(src_files)
# print("done")


/mnt/HDD1/shounak/Biopsy_output_year_sorted/17/anono14cff1mn/20170901/pet_1-_5_bp/prostate_diff_b800/MR.1.3.12.2.1107.5.2.38.51004.30000017092811190401000002726.13.dcm
/mnt/HDD1/shounak/Biopsy_output_year_sorted/17/anondphhef1rn/20160502/mrt_becken_nativ_und_mit_km/prostate_diff_b800/MR.1.3.12.2.1107.5.2.38.51004.30000017092811190401000002726.13.dcm
/mnt/HDD1/shounak/Biopsy_output_year_sorted/17/anon9u3cff1ci/20170731/mrt_becken_nativ_und_mit_km/prostate_diff_b800/MR.1.3.12.2.1107.5.2.38.51004.30000017092811190401000002726.13.dcm
/mnt/HDD1/shounak/Biopsy_output_year_sorted/17/anoni7hhef167/20161219/pet_1-_5_bp/prostate_diff_b800/MR.1.3.12.2.1107.5.2.38.51004.30000017092811190401000002726.13.dcm
/mnt/HDD1/shounak/Biopsy_output_year_sorted/17/anonff3cff1rj/20170901/pet_1-_5_bp/prostate_diff_b800/MR.1.3.12.2.1107.5.2.38.51004.30000017092811190401000002726.13.dcm
/mnt/HDD1/shounak/Biopsy_output_year_sorted/17/anondg5cff113/20170808/pet_1-_5_bp/prostate_diff_b800/MR.1.3.12.2.1107.5.2.38.510

In [ ]:
opfolder = "/mnt/HDD1/shounak/Biopsy_output"
filecount = []  #empty array which will store the file names
modalitycount = []
for root, dirs, files in os.walk(opfolder):
    for file in files: 
#         if ".file" in file:# exclude non-dicoms, good for messy folders
            filecount.append(os.path.join(root, file))
            for dicom_loc in filecount:
    # read the file
                ds1 = dicom.read_file(dicom_loc, force=True)
    
    # get patient, study, and series information
                modalitycount = ds1.get("Modality","NA")
#                 modalitycount.append(os.path.join(root, file))
print('%s files found.' % len(filecount))
print('modality count=', len(modalitycount))

In [ ]:
patientidlist = []
for root, dirs, files in os.walk(patientidlist):
    for file in files: 
#         if ".file" in file:# exclude non-dicoms, good for messy folders
            filecount.append(os.path.join(root, file))

print('%s files found.' % len(patientidlist))

for dicom_loc in filecount:
    # read the file
    ds1 = dicom.read_file(dicom_loc, force=True)
    
    # get patient, study, and series information
    mod = ds1.get("Modality","NA")
#     studyInstanceUID = ds1.get("StudyInstanceUID","NA")
#     seriesInstanceUID = ds1.get("SeriesInstanceUID","NA")
#     instanceNumber = str(ds1.get("InstanceNumber","0"))
    
    patientidlist.append(os.path.join(root, files))